In [4]:

import os, saspy, itertools, sys
import seaborn as sns
from tabulate import tabulate
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
today = datetime.today().strftime("%Y-%m-%d")
import geopy.distance
import pgeocode
dist = pgeocode.GeoDistance('US')
conv_fac = 0.621371

#Stats Import
from scipy.stats import wilcoxon 
from scipy.stats import shapiro 
from scipy.stats import kstest
# from numpy.random import seed
# from numpy.random import math

import pingouin as pg
from pingouin import pairwise_ttests, pairwise_tukey, pairwise_gameshowell
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import scipy.stats as stats
# from scipy import stats # Might need as is for ttest proportiality z or tdist
import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.formula.api import ols
from scipy.stats import wilcoxon
from statsmodels.stats.proportion import proportions_ztest



import io
from io import BytesIO
import base64


from operator import concat

# sys.path.append(os.path.join(sys.path[0],'C:\Junk\lamb_consulting'))
sys.path.append(os.path.join(sys.path[0],'U:\lamb_consulting'))


# import cox, means, regressions

from lamb_consulting import *

from lamb_consulting.means import means_desc, means_desc_class, aov
from lamb_consulting.freqs import freqs, chi_contingency, chi_oneway
from lamb_consulting.export import export_to_Excel
# from lamb_consulting.regressions import ols, regression_plot
from lamb_consulting.cox import cox
from lamb_consulting.kaplan_meier import km,km_plots


# For figure image to base 64 writing

def fig_to_base64(fig):
    img = io.BytesIO()
    fig.savefig(img, format='png',
                bbox_inches='tight')
    img.seek(0)

    return base64.b64encode(img.getvalue())


tpath = "U://Leach//Schnauzers_20230406//" 
output_path = tpath + "//output//"
fn="StdSchStats"
extension='.xlsx'
file = tpath+fn+extension
file
sch = pd.read_excel((file), sheet_name='Echo')

sch.columns= sch.columns.str.lower()

sch.reset_index(inplace=True)
sch = sch.rename(columns = {'index':'my_id'})

sch_col_l = sch.columns.tolist()

sch_col_l

drop_from_all_list = ['my_id', 'name', 'id', 'doe', 'dob', 'genotype', 'mitral_regurg', 'tricuspid_regur',
 'pulmonic_regurg', 'aortic_regurg', 'rhythm','conduction_block', 'sex']

# create new list using list comprehension
iv_cont_list = [i for i in sch_col_l if i not in drop_from_all_list]

iv_cat_list = ['sex', 'genotype']

cat_list = ['genotype', 'mitral_regurg', 'tricuspid_regur', 'pulmonic_regurg', 'aortic_regurg', 'rhythm','conduction_block', 'sex']


print(iv_cont_list)
print(iv_cat_list)
print(cat_list)

# h_nan_drop = h_nan_drop.rename(columns = {'home_club':'club_dup'}).reset_index()
# h_nan_drop = h_nan_drop.drop(columns=['index'])
# ha_nan.to_excel("_ha_nan.xlsx")

    

['age', 'weight', 'bsa', 'ntprobnp', 'ctni', 'aa_l', 'av_vmax', 'e_a_ratio', 'e_ea_ratio', 'ea_l', 'ea_aa_ratio', 'edv', 'edvi', 'ef', 'epss', 'esv', 'esvi', 'fs', 'ivrt', 'lvl', 'sphericity_index', 'ivsd', 'la_lax', 'nla_lax', 'la_sax', 'ao_sax', 'la_ao_ratio', 'lvidd', 'nlvidd', 'lvids', 'nlvids', 'lvpwd', 'mv_a', 'mv_e', 'pv_vmax', 'sv', 'heart_rate', 'mea']
['sex', 'genotype']
['genotype', 'mitral_regurg', 'tricuspid_regur', 'pulmonic_regurg', 'aortic_regurg', 'rhythm', 'conduction_block', 'sex']


# Start Groupings for mean, std, and other descriptive statistics



In [6]:
m_desc_df = pd.DataFrame()
m_class_df = pd.DataFrame()

# df_90_912p5 df_912p5_1825 df_1825_3560  df_90_3560
for i in iv_cont_list:
        desc_tmp = means_desc(sch, i,"sch")
        m_desc_df = m_desc_df.append(desc_tmp)


# Deliberately changing class to desc for appending for stacking purposes
for i in iv_cat_list:
    for j in iv_cont_list:
        desc_class_tmp = means_desc_class(sch,j, i,"df")
        m_class_df = m_class_df.append(desc_class_tmp)

print("m_desc_df")
display(m_desc_df.head())
print("m_class_df")
display(m_class_df.head())

m_desc_df


,df_used,cont_var,nmiss,n,mean,median,std,min,max,p25,p75
0,sch,age,0,66.0,4.230303,3.153425,3.015425,1.000000,11.742466,2.05411,4.986301
0,sch,weight,0,66.0,17.442424,17.250000,2.263426,13.100000,23.400000,16.00000,19.075000
0,sch,bsa,0,66.0,0.678046,0.674290,0.058698,0.561266,0.826289,0.64131,0.721048
0,sch,ntprobnp,0,66.0,410.606061,418.000000,182.150730,126.000000,948.000000,294.00000,483.250000
0,sch,ctni,0,66.0,22.969697,11.000000,32.178813,4.000000,172.000000,7.25000,21.500000


m_class_df


,df_used,cat_var,level,var,nmiss,n,mean,median,std,min,max,p25,p75
0,df,sex,2,age,0,29.0,4.323193,3.128767,3.110285,1.000000,11.742466,2.479452,6.630137
1,df,sex,1,age,0,37.0,4.157497,3.230137,2.980104,1.000000,11.715068,2.038356,4.920548
0,df,sex,2,weight,0,29.0,15.955172,15.900000,1.655637,13.500000,19.700000,14.400000,17.000000
1,df,sex,1,weight,0,37.0,18.608108,18.500000,1.984856,13.100000,23.400000,17.300000,20.300000
0,df,sex,2,bsa,0,29.0,0.639386,0.638635,0.043947,0.572634,0.736714,0.597810,0.667760


# Inferential two-way ANOVA for later upon specific request

In [ ]:


res = stat()
data_dict = {}
mt_data = pd.DataFrame()


ivs = iv_cat_list
df_evals = {1:sch}
dvs = dict(zip([*range(0, (len(iv_cont_list)), 1)], iv_cont_list))
# Set normality statistics
normal_on = 1
if normal_on != 0:
    f = open('my_normality.html','w')


# Set up dict for Full DVs Full Rank but Catgetoricals with No PPSN

for i in ivs: # independnet variable for xfac_var
    for k1,v1 in dvs.items():
        for k, v in df_evals.items():
            anova_df = v.loc[:,[v1,i]]
            print()
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)

                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                my_fig0 = plt
                encoded0 = fig_to_base64(my_fig0)
                my_html0 = '<img src="data:image/png;base64, {}">'.format(encoded0.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html0)

                # residuals histogram
                my_fig1 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig1.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                # encoded = base64.b64encode(tmpfile.getvalue()).decode('utf8') <img src='data:image/png;base64,{{}}'>
                my_html1 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html1)



            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])

# mt_data.to_excel("diffs_session.xlsx")  
mt_data.head()


In [ ]:

ivs = iv_cat_list
df_evals = {1:sch}
dvs = dict(zip([*range(0, (len(iv_cont_list)), 1)], iv_cont_list))
# Set normality statistics
normal_on = 1
if normal_on != 0:
    f = open('my_normality.html','w')
